# 线性回归模型，自定义梯度

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
print(tf.__version__)
print(sys.version_info)
for module in np, pd ,sklearn, tf, keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

# print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)


(20640, 8)
(20640,)


In [3]:
from sklearn.model_selection import train_test_split

x_train_all,x_test,y_train_all,y_test=train_test_split(housing.data, housing.target,random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all,y_train_all,random_state=11)

In [4]:
from sklearn.preprocessing import StandardScaler
transfer = StandardScaler()
x_train_scaled = transfer.fit_transform(x_train)
x_test_scaled = transfer.transform(x_test)
x_valid_scaled = transfer.transform(x_valid)

In [10]:
#metric的使用, 均方差
metric = keras.metrics.MeanSquaredError()
print(metric([5.],[2.]))
print(metric([0.],[1.]))
#之前结果会累加记录
print(metric.result())
#如果不想累加
metric.reset_states()#清空
metric([1.],[3.])
print(metric.result())

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [14]:
# 1、batch的形式遍历数据集 metric
#      1.1自动求导替换为手动改求导
# 2、epich结束，验证集验证 metric

epochs = 100
batch_size =32
step_per_epoch = len(x_test_scaled) //batch_size
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()
def random_batch(x,y, batch_size=32):
    idx = np.random.randint(0,len(x),size = batch_size)
    return x[idx], y[idx]



model = keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
    ])

for epoch in range(epochs):
    metric.reset_states()
    for step in range(step_per_epoch):
        x_batch, y_batch = random_batch(x_test_scaled, y_train, batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss  = tf.reduce_mean(
                    keras.losses.mean_squared_error(y_batch, y_pred))
            metric(y_batch,y_pred)
        grads = tape.gradient(loss, model.variables)
        grads_and_vars = zip(grads, model.variables )
        optimizer.apply_gradients(grads_and_vars)
        print('\r Epoch',epoch,'train mse:',metric.result().numpy(),end = ' ')
    y_valid_pred = model(x_valid_scaled)
    valid_loss  = tf.reduce_mean(
            keras.losses.mean_squared_error(y_valid_pred, y_valid))
    print('/t', 'valid mse: ', valid_loss.numpy())


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

 Epoch 0 train mse: 1.6900336 0 train mse: 1.8829952 1.732949  1.7156334 /t valid mse:  1.513169128094185
 Epoch 1 train mse: nan 341886e+27 5997 /t valid mse:  nan
 Epoch 2 train mse: nan train mse: nan /t valid mse:  nan
 Epoch 3 train mse: nan nan /t valid mse:  nan
 Epoch 4 train mse: nan /t valid mse:  nan
 Epoch 5 train mse: nan /t valid mse:  nan
 Epoch 6 train mse: nan /t valid mse:  nan
 Epoch 7 train mse: nan /t valid mse:  nan
 Epoch 8 train mse: nan train mse: nan  nan /t valid mse:  nan
 Ep

KeyboardInterrupt: 